In [0]:
import numpy as np

class Dequeue:
    def __init__(self):
        self.dq = []

    def insertFirst(self, item):
        self.dq.insert(0, item)

    def insertLast(self, item):
        self.dq.append(item)

    def isEmpty(self):
        if len(self.dq) > 0:
            return False
        else:
            return True

    def popFirst(self):
        return self.dq.pop(0)

    def popLast(self):
        return self.dq.pop(-1)

    def peekFirst(self):
        return self.dq[0]

    def peekLast(self):
        return self.dq[-1]

    def print(self):
        print(self.dq)

    def sum(self):
        return sum(self.dq)


def one_dim_median_filtering(image,window_size):
    filtered = np.zeros(image.shape)
    window_size = window_size
    gap = window_size // 2 # 제외되는 픽셀수

    for i in range(gap,image.shape[0]-gap):
        dq = Dequeue()
        for c in range(-gap,gap+1):
            for r in range(-gap,gap+1):
                dq.insertLast(image[i+r,1+c])
                filtered[i,1] = np.median(dq.dq)
        for j in range(2,image.shape[0]-1):
            for k in range(gap,-gap-1,-1):
                dq.popFirst()
                dq.insertLast(image[i+k,j+gap])
            filtered[i,j] = np.median(dq.dq)
    return filtered

def median_filtering(image,window_size = 3):
    result = []
    for i in range(image.shape[2]):
        result.append(one_dim_median_filtering(image[:,:,i],window_size))
    result = np.stack(result,axis=2)
    return result

from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image

class Window:
    def __init__(self):
        self.window=tk.Tk()
        self.window.title("Image Viewer")
        self.window.geometry("640x480")
        self.window.resizable(False, False)

        # 메뉴바를 윈도우에 추가한다.
        menubar = tk.Menu(self.window)
        menu_1 = tk.Menu(menubar, tearoff=0)
        menu_1.bind('<<MenuSelect>>')

        menu_1.add_command(label="Open", command=self.fileSelect)
        menubar.add_cascade(label="File", menu=menu_1)

        self.window.config(menu=menubar)

        # 이미지 라벨을 추가한다.
        self.imgLabel = tk.Label(self.window, width=400, height=400, relief='solid')
        self.imgLabel.pack()

        self.window.mainloop()

    # 메뉴에서 close 가 선택되었을 때 수행한다.
    def close(self):
        self.window.quit()
        self.window.destroy()

    # 파일을 선택한다.
    def fileSelect(self):
        # 초기 디렉토리를 루트로 지정하고 파일을 선택하면 해당 파일명을 selFile에 입력한다.
        selFile = filedialog.askopenfilename(initialdir="/", title="Select file", filetypes=(("image files", "*.jpg *.png"), ("all files", "*.*")))
        print(selFile)
        self.image = Image.open(selFile)
        print(self.image.size[0], self.image.size[1])
        # 해당 이미지의 크기를 400, 400으로 resize 한다.
        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))
        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.image = np.array(self.image)
        self.image = (median_filtering(self.image,window_size = 3))
        self.image = ImageTk.PhotoImage(image=Image.fromarray(self.image.astype('uint8')))
        self.imgLabel.config(image = self.image)
a = Window()